# Case Study 2: Filter Pushdown

> `Filter pushdown` improves performance by reducing the amount of data shuffled during any dataframes transformations.

### Library Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Create a `SparkSession`. No need to create `SparkContext` as you automatically get it as part of the `SparkSession`.

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Exploring Joins") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext

### Initial Datasets

In [3]:
df_1 = spark.createDataFrame(
    [
        (1, 1, 'a'), 
        (2, 1, 'b'), 
        (2, 2, 'c'), 
    ], ['shop_id', 'data_id', 'val_1']
)

df_1.toPandas()

,shop_id,data_id,val_1
0,1,1,a
1,2,1,b
2,2,2,c


In [4]:
df_2 = spark.createDataFrame(
    [
        (1, 1, 10), 
        (2, 2, 20), 
    ], ['shop_id', 'data_id', 'val_2']
)

df_2.toPandas()

,shop_id,data_id,val_2
0,1,1,10
1,2,2,20


## Option #1: Join the data, then perform Filter

In [5]:
df = df_1 \
    .join(df_2.drop('shop_id'), 'data_id') \
    .filter(F.col('shop_id') == 1)

df.toPandas()

,data_id,shop_id,val_1,val_2
0,1,1,a,10


In [6]:
df.explain()

== Physical Plan ==
*(5) Project [data_id#1L, shop_id#0L, val_1#2, val_2#8L]
+- *(5) SortMergeJoin [data_id#1L], [data_id#7L], Inner
   :- *(2) Sort [data_id#1L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(data_id#1L, 200)
   :     +- *(1) Filter ((isnotnull(shop_id#0L) && (shop_id#0L = 1)) && isnotnull(data_id#1L))
   :        +- Scan ExistingRDD[shop_id#0L,data_id#1L,val_1#2]
   +- *(4) Sort [data_id#7L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(data_id#7L, 200)
         +- *(3) Project [data_id#7L, val_2#8L]
            +- *(3) Filter isnotnull(data_id#7L)
               +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened:**

* We can see that the filter is only done on one side of the join. 
* This means all of the data is brough to the join.

**Results:**

We bring more data to the `join` and `shuffle`, **this is bad**.

## Option #2: Join on Filter Key, then Filter

In [7]:
df = df_1 \
    .join(df_2, ['shop_id', 'data_id']) \
    .filter(F.col('shop_id') == 1)

df.toPandas()

,shop_id,data_id,val_1,val_2
0,1,1,a,10


In [8]:
df.explain()

== Physical Plan ==
*(5) Project [shop_id#0L, data_id#1L, val_1#2, val_2#8L]
+- *(5) SortMergeJoin [shop_id#0L, data_id#1L], [shop_id#6L, data_id#7L], Inner
   :- *(2) Sort [shop_id#0L ASC NULLS FIRST, data_id#1L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(shop_id#0L, data_id#1L, 200)
   :     +- *(1) Filter ((isnotnull(shop_id#0L) && (shop_id#0L = 1)) && isnotnull(data_id#1L))
   :        +- Scan ExistingRDD[shop_id#0L,data_id#1L,val_1#2]
   +- *(4) Sort [shop_id#6L ASC NULLS FIRST, data_id#7L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(shop_id#6L, data_id#7L, 200)
         +- *(3) Filter ((isnotnull(shop_id#6L) && isnotnull(data_id#7L)) && (shop_id#6L = 1))
            +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened:**
* A `filter` was performed on both datasets before the join.
* Less data is brought to the join and shuffle.
* We `shuffle`d on 2 keys (`shop_id, data_id`).

**Results:**
* We bring less data to the `join` and `shuffle`, **this is good**.
* We `shuffle`d on more keys.

## Option #3: Filter Left, then Join

In [9]:
df = df_1 \
    .filter(F.col('shop_id') == 1) \
    .join(df_2.drop('shop_id'), 'data_id')

df.toPandas()

,data_id,shop_id,val_1,val_2
0,1,1,a,10


In [10]:
df.explain()

== Physical Plan ==
*(5) Project [data_id#1L, shop_id#0L, val_1#2, val_2#8L]
+- *(5) SortMergeJoin [data_id#1L], [data_id#7L], Inner
   :- *(2) Sort [data_id#1L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(data_id#1L, 200)
   :     +- *(1) Filter ((isnotnull(shop_id#0L) && (shop_id#0L = 1)) && isnotnull(data_id#1L))
   :        +- Scan ExistingRDD[shop_id#0L,data_id#1L,val_1#2]
   +- *(4) Sort [data_id#7L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(data_id#7L, 200)
         +- *(3) Project [data_id#7L, val_2#8L]
            +- *(3) Filter isnotnull(data_id#7L)
               +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened:**
* This is exactly the same as case 1.

## Option #4: Filter Both, then Join

In [11]:
df_3 = df_2 \
    .filter(F.col('shop_id') == 1) \
    .drop('shop_id')

df = df_1 \
    .filter(F.col('shop_id') == 1) \
    .join(df_3, 'data_id')

df.toPandas()

,data_id,shop_id,val_1,val_2
0,1,1,a,10


In [12]:
df.explain()

== Physical Plan ==
*(5) Project [data_id#1L, shop_id#0L, val_1#2, val_2#8L]
+- *(5) SortMergeJoin [data_id#1L], [data_id#7L], Inner
   :- *(2) Sort [data_id#1L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(data_id#1L, 200)
   :     +- *(1) Filter ((isnotnull(shop_id#0L) && (shop_id#0L = 1)) && isnotnull(data_id#1L))
   :        +- Scan ExistingRDD[shop_id#0L,data_id#1L,val_1#2]
   +- *(4) Sort [data_id#7L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(data_id#7L, 200)
         +- *(3) Project [data_id#7L, val_2#8L]
            +- *(3) Filter ((isnotnull(shop_id#6L) && (shop_id#6L = 1)) && isnotnull(data_id#7L))
               +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened:**
* We performed a `filter` on both datasets before the join.
* We shuffled on only one key (`data_id`).

**Results:**
* We bring less data to the `join` and `shuffle`, **this is good**.
* We only `shuffle`d on 1 key, **this is good**.

## TL;DR

**Option #2** (Good)
* When we `join` on the `filter` key (`shop_id`) as well, this caused a `filter` on both datasets prior to the join, this is called a `filter-pushdown`.
* By joining on both both keys, this made us `sort` on 2 keys.

**Option #4** (Better)
* When we pre `filter` both datasets, this caused a `filter-pushdown`.
* We only `join` and `sort` on one key as well, which is good as well.

## All to Say

* We should always try to push the `filter` down as much as possible.
* This means that there will be less data being `shuffled` and brought to the `join`.
* This can be achieved with join in case #2 or #4.

**Note:** This might only works if the `filter key` is on both sides, might not work in all situations.